#### Active learning NNGP experiment
multiple params

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random

import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt

from model.mlp import MLP
from dataloader.rosen import RosenData
from uncertainty_estimator.nngp import NNGP
from uncertainty_estimator.mcdue import MCDUE
from uncertainty_estimator.random_estimator import RandomEstimator
from sample_selector.eager import EagerSampleSelector
from oracle.identity import IdentityOracle
from al_trainer import ALTrainer

import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False


In [ ]:
config = {
    'random_seed': 4623457,
    'n_dim': 10,
    'n_train': 200,
    'n_test': 200,
    'n_pool': 1000,
    'layers': [128, 64, 32],
    'update_sample_size': 100,
    'al_iterations': 10
}

if config['random_seed'] is not None:
    np.random.seed(config['random_seed'])
    random.seed(config['random_seed'])

In [ ]:
def print_shapes(note, *sets):
    print(note)
    for x, y in sets:
        print("shapes:", x.shape, y.shape)

# load data
X_train, y_train, _, _, X_test, y_test, X_pool, y_pool = RosenData(
    config['n_train'], 200, config['n_test'], config['n_pool'], config['n_dim']
).dataset(use_cache=True)

In [ ]:
# Init neural network & tf session
tf.reset_default_graph()
if config['random_seed'] is not None:
    tf.set_random_seed(config['random_seed'])

model = MLP(
    ndim = config['n_dim'],
    random_state = config['random_seed'],
    layers = config['layers']
)

try:
    sess.close()
except:
    pass


init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

model.set_session(sess)

In [ ]:
estimator = NNGP(model)  # to estimate uncertainties
estimator.set_session(sess)
oracle = IdentityOracle(y_pool)  # generate y for X from pool
sampler = EagerSampleSelector(oracle) # sample X and y from pool by uncertainty estimations

# estimator.estimate(X_train, y_train, X_pool)[:40]

In [ ]:
trainer = ALTrainer(model, estimator, sampler, oracle, config['al_iterations'], config['update_sample_size'])
rmses = trainer.train(X_train, y_train, X_test, y_test, X_test, y_test, X_pool)


In [ ]:
print(rmses)
plt.plot(rmses, label='NNGP', marker='.')
plt.title('RMS Error by active learning iterations')
plt.legend()

In [ ]:
# Show some predictions
print(model.predict(data = X_test[:3]))
print(y_test[:3])

In [ ]:
# show some uncertainties
estimator.estimate(X_train, y_train, X_pool)[:10]